In [1]:
import numpy as np
import pandas as pd

In [21]:
import json
import numpy as np
import pandas as pd


def data_processing_linear_regression(filename, non_invertible, mapping, mapping_power):
  

  white = pd.read_csv(filename, low_memory=False, sep=';').values

  [N, d] = white.shape

  if(mapping == True):
    maped_X = mapping_data(white[:,:-1],mapping_power)
    white = np.insert(maped_X, maped_X.shape[1], white[:,-1], axis=1)
    

  np.random.seed(3)
  # prepare data
  ridx = np.random.permutation(N)
  ntr = int(np.round(N * 0.8))
  nval = int(np.round(N * 0.1))
  ntest = N - ntr - nval

  # spliting training, validation, and test

  Xtrain = np.hstack([np.ones([ntr, 1]), white[ridx[0:ntr], 0:-1]])

  ytrain = white[ridx[0:ntr], -1]

  Xval = np.hstack([np.ones([nval, 1]), white[ridx[ntr:ntr + nval], 0:-1]])
  yval = white[ridx[ntr:ntr + nval], -1]

  Xtest = np.hstack([np.ones([ntest, 1]), white[ridx[ntr + nval:], 0:-1]])
  ytest = white[ridx[ntr + nval:], -1]
  if(non_invertible == True):
    N, D = Xtrain.shape
    np.random.seed(4)
    random_row = np.random.randint(N)
    random_col = np.random.randint(D)
    
    Xtrain[:,random_col] = 0
    Xtrain[random_row,:] = 0
    
    return Xtrain, ytrain, Xval, yval, Xtest, ytest
  return Xtrain, ytrain, Xval, yval, Xtest, ytest


In [32]:
def mean_absolute_error(w, X, y):
    """
    Compute the mean absolute error on test set given X, y, and model parameter w.
    Inputs:
    - X: A numpy array of shape (num_samples, D) containing test feature.
    - y: A numpy array of shape (num_samples, ) containing test label
    - w: a numpy array of shape (D, )
    Returns:
    - err: the mean absolute error
    """
    #####################################################
    # TODO 1: Fill in your code here #
    #####################################################
    err = None
    num_samples = y.shape[0]
    err = np.sum(abs(X.dot(w) - y))/num_samples
    return err

In [33]:
def linear_regression_noreg(X, y):
    w = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)
    return w

In [34]:
filename = 'winequality-white.csv'

print("\n======== Question 1.1 and Question 1.2 ========")
Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing_linear_regression(filename, False, False, 0)
w = linear_regression_noreg(Xtrain, ytrain)
print("dimensionality of the model parameter is ", w.shape, ".", sep="")
print("model parameter is ", np.array_str(w))
mae = mean_absolute_error(w, Xtrain, ytrain)
print("MAE on train is %.5f" % mae)
mae = mean_absolute_error(w, Xval, yval)
print("MAE on val is %.5f" % mae)
mae = mean_absolute_error(w, Xtest, ytest)
print("MAE on test is %.5f" % mae)


======== Question 1.1 and Question 1.2 ========
dimensionality of the model parameter is (12,).
model parameter is  [ 2.03721116e+02  1.09955585e-01 -1.93164831e+00 -4.90845226e-02
  1.02194195e-01 -5.45232539e-02  4.00189586e-03  1.52537227e-04
 -2.04673020e+02  9.04608186e-01  6.41578532e-01  1.32320100e-01]
MAE on train is 0.58020
MAE on val is 0.59410
MAE on test is 0.56079


In [78]:
###### Q1.3 ######
def linear_regression_invertible(X, y):
    """
    Compute the weight parameter given X and y.
    Inputs:
    - X: A numpy array of shape (num_samples, D) containing feature.
    - y: A numpy array of shape (num_samples, ) containing label
    Returns:
    - w: a numpy array of shape (D, )
    """
    #####################################################
    # TODO 3: Fill in your code here #
    #####################################################
    w = None
    cnt = 1
    XTX = X.T.dot(X)
    while(abs(np.min(np.linalg.eigvals(XTX + cnt * np.eye(XTX.shape[0]))))< pow(10, -5)):
        cnt += 1
    XTX = XTX + cnt * np.eye(XTX.shape[0])
    w = np.linalg.inv(XTX).dot(X.T).dot(y)
    return w

In [79]:
print("\n======== Question 1.3 ========")
Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing_linear_regression(filename, True, False, 0)
w = linear_regression_invertible(Xtrain, ytrain)
print("dimensionality of the model parameter is ", w.shape, ".", sep="")
print("model parameter is ", np.array_str(w))
mae = mean_absolute_error(w, Xtrain, ytrain)
print("MAE on train is %.5f" % mae)
mae = mean_absolute_error(w, Xval, yval)
print("MAE on val is %.5f" % mae)
mae = mean_absolute_error(w, Xtest, ytest)
print("MAE on test is %.5f" % mae)


======== Question 1.3 ========
dimensionality of the model parameter is (12,).
model parameter is  [ 0.84355506 -0.04441418 -2.00890117 -0.11681133  0.0285244  -0.48792001
  0.00431189  0.          0.66002924  0.24521302  0.350988    0.38109447]
MAE on train is 0.58522
MAE on val is 0.60007
MAE on test is 0.55819


In [80]:
###### Q1.4 ######
def regularized_linear_regression(X, y, lambd):
    """
    Compute the weight parameter given X, y and lambda.
    Inputs:
    - X: A numpy array of shape (num_samples, D) containing feature.
    - y: A numpy array of shape (num_samples, ) containing label
    - lambd: a float number containing regularization strength
    Returns:
    - w: a numpy array of shape (D, )
    """
  #####################################################
  # TODO 4: Fill in your code here #
  #####################################################		
    w = None
    XTX = X.T.dot(X)
    XTX += lambd * np.eye(XTX.shape[0])
    w = np.linalg.inv(XTX).dot(X.T).dot(y)
    return w


In [81]:
print("\n======== Question 1.4 ========")
Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing_linear_regression(filename, True, False, 0)
w = regularized_linear_regression(Xtrain, ytrain, 0.1)
print("dimensionality of the model parameter is ", w.shape, ".", sep="")
print("model parameter is ", np.array_str(w))
mae = mean_absolute_error(w, Xtrain, ytrain)
print("MAE on train is %.5f" % mae)
mae = mean_absolute_error(w, Xval, yval)
print("MAE on val is %.5f" % mae)
mae = mean_absolute_error(w, Xtest, ytest)
print("MAE on test is %.5f" % mae)


======== Question 1.4 ========
dimensionality of the model parameter is (12,).
model parameter is  [ 1.72490981 -0.04898016 -2.05742338 -0.12068529  0.02838304 -0.81308859
  0.00425276  0.          0.00675103  0.20653332  0.35756603  0.37653797]
MAE on train is 0.58509
MAE on val is 0.59939
MAE on test is 0.55777


In [60]:
###### Q1.5 ######
def tune_lambda(Xtrain, ytrain, Xval, yval):
    """
    Find the best lambda value.
    Inputs:
    - Xtrain: A numpy array of shape (num_training_samples, D) containing training feature.
    - ytrain: A numpy array of shape (num_training_samples, ) containing training label
    - Xval: A numpy array of shape (num_val_samples, D) containing validation feature.
    - yval: A numpy array of shape (num_val_samples, ) containing validation label
    Returns:
    - bestlambda: the best lambda you find in lambds
    """
    #####################################################
    # TODO 5: Fill in your code here #
    #####################################################		
    bestlambda = None
    mae = float('inf')
    for lamda in range(-19, 20):
        w = regularized_linear_regression(Xtrain, ytrain, pow(10, lamda))
        if mean_absolute_error(w, Xval, yval) < mae:
            bestlambda = pow(10, lamda)
    return bestlambda

In [59]:
print("\n======== Question 1.5========")
Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing_linear_regression(filename, False, False, 0)
bestlambd = tune_lambda(Xtrain, ytrain, Xval, yval)
print("Best Lambda =  " + str(bestlambd))
w = regularized_linear_regression(Xtrain, ytrain, bestlambd)
print("dimensionality of the model parameter is ", len(w), ".", sep="")
print("model parameter is ", np.array_str(w))
mae = mean_absolute_error(w, Xtrain, ytrain)
print("MAE on train is %.5f" % mae)
mae = mean_absolute_error(w, Xval, yval)
print("MAE on val is %.5f" % mae)
mae = mean_absolute_error(w, Xtest, ytest)
print("MAE on test is %.5f" % mae)


======== Question 1.5========
Best Lambda =  0.1
dimensionality of the model parameter is 12.
model parameter is  [ 1.67492616e+00 -4.46761693e-02 -2.00550032e+00 -1.08678263e-01
  2.92199160e-02 -7.64112758e-01  5.23116206e-03 -7.54594811e-04
  5.17634041e-02  2.30856732e-01  3.78135271e-01  3.70053691e-01]
MAE on train is 0.58255
MAE on val is 0.60029
MAE on test is 0.55552
